Unseen query to templatize:
1.
SELECT DISTINCT ?var1 WHERE { ?var2 ?var1 ?var3 . ?var2 <http://www.wikidata.org/prop/direct/P705> ?var4 . }

2.
PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?var1 ?var2 WHERE { ?var1 wdt:P982 ?var2 }

3.
SELECT DISTINCT ?var1 WHERE { ?var2 ?var1 ?var3 . ?var2 <http://www.wikidata.org/prop/direct/P682> ?var4 . }

4.
PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT * WHERE { ?var1 wdt:P3230 ?var2 }

5.
SELECT * WHERE { ?var1 ?var2 ?var3 . FILTER ( ( REGEX ( ?var3 , "caledfwlch", "i" ) ) ) . } 
Also with "P1559"

6.
SELECT ?var1 ?var2 ?var3 WHERE { ?var1 ?var2 ?var3 . FILTER ( ( REGEX ( STR ( ?var1 ) , "alice" ) ) ) . }


In [ ]:
import json
import json5
import os
pred_freq_path = '/PlanRGCN/data/wikidata/predicate/pred_stat/batches_response_stats/freq'
pred_freq = {}
freq_pred = {}
for i in os.listdir(pred_freq_path):
    with open(os.path.join(pred_freq_path, i),'r') as f:
        try:
            data = json.load(f)
        except:
            try:
                data = json5.load(f)
            except:
                continue
        data = data['results']['bindings']
        for v in data:
            pred_freq[v['p1']['value']] = int(v['triples']['value'])
            try:
                freq_pred[int(v['triples']['value'])].append(v['p1']['value'])
            except:
                freq_pred[int(v['triples']['value'])] = [v['p1']['value']]
                
    

In [ ]:
q1 = f'''
SELECT DISTINCT ?var1 WHERE 
{{ ?var2 ?var1 ?var3 . 
    ?var2 <http://www.wikidata.org/prop/direct/P705> ?var4 . 
}}
'''

q2 = f'''
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
SELECT ?var1 ?var2 WHERE {{ ?var1 wdt:P982 ?var2 }}
'''

q4 = f'''
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
SELECT * WHERE {{ ?var1 wdt:P3230 ?var2 }}
'''

q5 = f'''
SELECT * WHERE {{ 
?var1 ?var2 ?var3 . 
FILTER ( ( REGEX ( ?var3 , "caledfwlch", "i" ) ) ) . }} 
'''
q6 = f'''
SELECT ?var1 ?var2 ?var3 
WHERE {{ ?var1 ?var2 ?var3 . 
FILTER ( ( REGEX ( STR ( ?var1 ) , "alice" ) ) ) . }}
'''

Analysis using isql

## Model Inference

In [6]:
import os
query = "SELECT * WHERE {?s ?p ?o}"
jarfile = '/PlanRGCN/PlanRGCN/qpe/target/qpe-1.0-SNAPSHOT.jar'
tempf = '/tmp/q.json'
cmd = f'java -jar {jarfile} extract-single \"{query}\" {tempf}'
os.system(cmd)

(bgp (triple ?s ?p ?o))


0

In [26]:
import pickle as p
prep_path ='/data/wikidata_0_1_10_v3_path_weight_loss/planrgcn/prepper.pcl'
prep_path ='/prepper.pcl'
with open(prep_path,'rb') as f:
    prep = p.load(f)

In [27]:
featurizer = prep.feat

In [8]:
import torch
model_path = '/data/wikidata_0_1_10_v3_path_weight_loss/planrgcn_binner_litplan/best_model.pt'
model =torch.load(model_path)

In [24]:
from trainer.model import Classifier2RGCN
inputd = 415
l1=2048
l2=4096
dropout= 0.0
n = 3
model = Classifier2RGCN(inputd, l1, l2,dropout, n)
model.load_state_dict(torch.load(model_path)['model_state'])

<All keys matched successfully>

In [ ]:
model

In [28]:
import json
from graph_construction.qp.query_plan_path import QueryPlanPath
try:
    data = json.load(open(tempf,'r'))
    qp = QueryPlanPath(data)
    qp.feature(featurizer)
    dgl_graph = qp.to_dgl()
    feats = dgl_graph.ndata["node_features"]
    edge_types = dgl_graph.edata["rel_type"]
    pred = model(dgl_graph, feats, edge_types)
    #print(pred)
except Exception as e:
    print(e)

mat1 and mat2 shapes cannot be multiplied (0x438 and 415x2048)


In [ ]:
prep_path ='/data/wikidata_0_1_10_v3_path_weight_loss/planrgcn/prepper.pcl'
with open(prep_path
featurizer = # load featurizer
model = #load model
try:
    data = json.load(open(tempf,'r'))
    qp = QueryPlanPath(data)
    qp.feature(featurizer)
    dgl_graph = qp.to_dgl()
    feats = dgl_graph.ndata["node_features"]
    edge_types = dgl_graph.edata["rel_type"]
    pred = model(dgl_graph, feats, edge_types)
    print(pred)
except:
    pass